In [ ]:
import convert_morph as conv 
import parse_conllu as parser
from pathlib import Path

import pandas as pd
import re 



In [ ]:
# Konverter alle splittene av retokenisert ndt data

for part in ["train", "test", "dev"]: 
    for lang in ["nb", "nn"]: 
        fpath = Path(f"data/ndt_{lang}_{part}.conllu")
        data = parser.parse_conll_file(fpath)
        morphdata = conv.convert_morphology(data)
        output = fpath.parent / f"{fpath.stem}_udmorph{fpath.suffix}"
        parser.write_conll(morphdata, output, add_comments=True)
        !udapy -s ud.SetSpaceAfterFromText < $output > out.conllu && mv out.conllu $output

---

In [ ]:
# Datautvalg
gullfil = Path("data/gullkorpus/2023_gullkorpus_ud.conllu")
ud_pregullfil = Path("data/gullkorpus/2019_gullkorpus_ud_før_annotasjon.conllu")
ndt_gullfil = Path("data/gullkorpus/2019_gullkorpus_ndt.conllu")

part = "dev" 
#part = "train"
#part = "test"
lang="nb"
#lang="nn"
ndt_fil = Path(f"data/ndt_{lang}_{part}.conllu")



In [ ]:
# Last inn data 
fpath = ndt_fil
data = parser.parse_conll_file(fpath)

In [ ]:
# Konverter UPOS og FEATS til UD-standard
morphdata = conv.convert_morphology(data)

In [ ]:
# Undersøk resultatet i pandas
df = parser.load_conll_to_df(morphdata)

In [ ]:
df

In [ ]:

outputfile = (fpath.parent / f"{fpath.stem}_udmorph{fpath.suffix}")
parser.write_df_to_conll(df, outputfile, add_comments=True)

---

In [ ]:
# Partisjoner data i dev, train, og test-splittene som UD er delt inn i
partitions = {
    "test" :  parser.filereadlines("data/test_ids.txt"),
    "dev" : parser.filereadlines("data/dev_ids.txt"),
    "train" : parser.filereadlines("data/train_ids.txt"),
}

for part, part_ids in partitions.items():
    outputfile = (fpath.parent / f"{fpath.stem}_{part}{fpath.suffix}")
    part_df = df[df.sent_id.isin(part_ids)]
    parser.write_df_to_conll(part_df, outputfile, add_comments=True)

In [ ]:
#Hent ut setningene som gir feilmelding 

fpath = Path("validation-report_ndt2ud.txt")
etype = "rel-upos-punct"

rows = Path(fpath).read_text(encoding="utf-8").splitlines()

error_info_regx = re.compile(r"^\[Line (\d+)(?: Sent )?(\d+)?(?: Node )?(\d+)?\]\: \[(L.*)\] (.*)(\[[0-9]*, [0-9]*\])?(.*)?$", flags=re.DOTALL)
errors = []
for row in rows:
    m = error_info_regx.fullmatch(row)
    if m is not None:
        errors.append(m.groups())

df = pd.DataFrame(errors, columns=["line", "sent","node", "errortype", "message", "relevant_nodes", "message2"])

errorlines = df[df.errortype.str.contains(etype)]

type_counts = df.errortype.value_counts()
print(type_counts)



In [ ]:
ndt = Path("data/ndt_nb_train_udmorph.conllu")

data = parser.load_conll_to_df(parser.parse_conll_file(ndt))
error_sents = data[data.sent_id.isin(errorlines["sent"])]

parser.write_df_to_conll(error_sents, f"error_{etype}_sents.conllu", 
                         add_comments=True)


In [ ]:
error_sents.sent_id.nunique()